In [3]:
%%time
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
import tools.Sample_Tools as smpl
import Analysis_Funs as af
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import talib
import inspect

from tools.Test_Tools import FactorTest
import tools.Pretreat_Tools as pretreat
from base.JuUnits import excute_for_multidates

from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date)

import tools.Factor_Cacher as cacher


import toys
%load_ext autoreload
%autoreload 2
%aimport tools.Factor_Cacher



ind_fun = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
data = make_data(zs='沪深300',end='2021-11-28', gap=2500)
for i in ind_fun:
    fun_name = 'alpha'+str(i)
    params = inspect.signature(getattr(toys, fun_name)).parameters.keys()
    indx = excute_for_multidates(data, lambda x: getattr(toys, fun_name)(*[x[param].copy() for param in params]) ,level=1)
    indx.name = fun_name
    cacher.save_cache(fun_name,indx)
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Wall time: 13min 47s


In [2]:
def make_data(zs='上证50',end='2021-11-28', gap=250):
    data = smpl.get_sample_by_zs(name=zs, end=end, gap=gap,  only_main=True, filter_st=True)
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data.data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)


    close_ind = pretreat.neutralize(data.close, data['ind'],categorical=['ind'])
    close_ind.name = 'close_ind'
    vwap_ind = pretreat.neutralize(data.vwap, data['ind'],categorical=['ind'])
    vwap_ind.name = 'vwap_ind'
    high_ind = pretreat.neutralize(data.high, data['ind'],categorical=['ind'])
    high_ind.name = 'high_ind'
    low_ind = pretreat.neutralize(data.low, data['ind'],categorical=['ind'])
    low_ind.name = 'low_ind'
    volume_ind = pretreat.neutralize(data.volume, data['ind'],categorical=['ind'])
    volume_ind.name = 'volume_ind'

    adv20 = excute_for_multidates(data.volume, lambda x:x.rolling(20).agg('mean'), level=1)
    adv20 = pd.concat([adv20,data['ind']],axis=1).dropna()
    adv20_ind = pretreat.neutralize(adv20.volume, adv20['ind'],categorical=['ind'])
    adv20_ind.name = 'adv20_ind'

    adv40 = excute_for_multidates(data.volume, lambda x:x.rolling(40).agg('mean'), level=1)
    adv40 = pd.concat([adv40, data['ind']],axis=1).dropna()
    adv40_ind = pretreat.neutralize(adv40.volume, adv40['ind'],categorical=['ind'])
    adv40_ind.name = 'adv40_ind'

    adv81 = excute_for_multidates(data.volume, lambda x:x.rolling(81).agg('mean'), level=1)
    adv81 = pd.concat([adv81, data['ind']],axis=1).dropna()
    adv81_ind = pretreat.neutralize(adv81.volume, adv81['ind'],categorical=['ind'])
    adv81_ind.name = 'adv81_ind'

    co_mixed = ((data.close * 0.60733) + (data.Open * (1 - 0.60733)))
    co_mixed_ind = pretreat.neutralize(co_mixed, data['ind'],categorical=['ind'])
    co_mixed_ind.name = 'co_mixed_ind'

    oh_mixed = ((data.Open * 0.868128) + (data.high * (1 - 0.868128)))
    oh_mixed_ind = pretreat.neutralize(oh_mixed, data['ind'],categorical=['ind'])
    oh_mixed_ind.name = 'oh_mixed_ind'

    lv_mixed = ((data.low * 0.721001) + (data.vwap * (1 - 0.721001)))
    lv_mixed_ind = pretreat.neutralize(lv_mixed, data['ind'],categorical=['ind'])
    lv_mixed_ind.name = 'lv_mixed_ind'

    return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)




In [4]:
%%time

def check_quality(ind_binned_ret):
    x = ind_binned_ret
    # x = x/np.abs(x).max()+0.000001
    ran = x.rank(ascending=False)
    # print(ran.values)
    r_cor = np.corrcoef(ran,x.index)[0][1]
    # print(r_cor,x.idxmin(),x.idxmax())
    
    if np.abs(r_cor)>0.45:
        if x.idxmax() in [1,2]:
            return 'left'
        if x.idxmax() in [9,10]:
            return 'right'
        return None
    
    if np.abs(r_cor) < 0.16:
        if x.idxmax() in [5,6]:
            return 'sem_mid'
        if x.idxmax() in [1,10]:
            x[x.idxmax()]=0
            if x.idxmax() in [1,10]:
                return 'sem_bilateral'
        return None

    return None


def cal_quality(data_df, Class, fun_name):    
    params = inspect.signature(getattr(Class, fun_name)).parameters.keys()
    
    indx = excute_for_multidates(data, lambda x: getattr(Class, fun_name)(*[x[param].copy() for param in params]) ,level=1)
    indx.name = fun_name
    tes = FactorTest(target_field=fun_name)
    tes.process_ind(pd.DataFrame(indx),data.ret_forward,simple=True)
    try:
        x = tes.get_ind_binned_ret_avg()
    except :
        # with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.width', 999):
        #     print(tes.ind_binned)
        # print('\r\n')
        return None
    
    if len(x) == 0:
        # print('errrrrrrorlen(x) == 0\r\n')
        return None
    return check_quality(x)

def cal_all_fun(data_df, Class):
    qs=[]
    idx=[]
    for i in range(1,102):
        fun_name = 'alpha'+str(i)
        qs.append(cal_quality(data_df, Class, fun_name))
        idx.append(fun_name)
    return pd.Series(qs,index=idx)

res = []
idxs = []
for year in [2021,2020,2019,2018,2017,2016,2015,2014,2013]:
    print('start',year)
    data = make_data(zs='沪深300',end='%d-11-28'%(year), gap=250)
    res.append(cal_all_fun(data, toys))
    idxs.append(year)
res_df = pd.concat(res, axis=1)
res_df.columns = idxs

res_df.to_excel('./300.xlsx')

res = []
idxs = []
for i in range(37):
    dt = QA_util_get_pre_trade_date('2021-12-28', 60*i)
    print('start',dt)
    data = make_data(zs='沪深300',end=dt, gap=125)
    res.append(cal_all_fun(data, toys))
    idxs.append(dt)

res_df = pd.concat(res, axis=1)
res_df.columns = idxs
# with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.width', 999):
#     print(res_df)
res_df.to_excel('./300-season.xlsx')

start 2021
start 2020
start 2019
start 2018
start 2017
start 2016
start 2015
start 2014
start 2013
start 2021-12-28
start 2021-09-28
start 2021-07-05
start 2021-04-06
start 2021-01-04
start 2020-10-09
start 2020-07-09
start 2020-04-09
start 2020-01-07
start 2019-10-14
start 2019-07-12
start 2019-04-15
start 2019-01-11
start 2018-10-17
start 2018-07-17
start 2018-04-19
start 2018-01-16
start 2017-10-23
start 2017-07-24
start 2017-04-26
start 2017-01-23
start 2016-10-28
start 2016-07-27
start 2016-04-29
start 2016-01-28
start 2015-11-04
start 2015-08-03
start 2015-05-08
start 2015-02-04
start 2014-11-10
start 2014-08-08
start 2014-05-15
start 2014-02-17
start 2013-11-15
start 2013-08-14
start 2013-05-17
start 2013-02-08
Wall time: 3h 27min 19s


In [593]:
ppp = inspect.signature(getattr(toys, 'alpha19')).parameters.keys()
aa = excute_for_multidates(data, lambda x: getattr(toys, 'alpha19')(*[x[param].copy() for param in ppp]) ,level=1)
aa.name = 'alpha19'
tt = FactorTest(target_field='alpha19')
tt.process_ind(pd.DataFrame(aa),data.ret_forward,simple=True)
tt.ind_ret_df

,,alpha19,ret_forward
date,code,,


In [786]:
# for year in [2021,2020,2019,2018,2017,2016,2015,2014,2013]:
#     print('start',year)
#     data = make_data(zs='沪深300',end='%d-11-28'%(year), gap=120)
# smpl.get_data(['600000'], start=None, end=None, gap=60, freq=QA.FREQUENCE.DAY, market=MARKET_TYPE.STOCK_CN)
# [0:7]

# from sklearn import preprocessing
preprocessing.normalize(np.array([i for i in range(1, 21)]).reshape(-1, 1), norm='l1', axis=1).reshape(-1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])